In [ ]:
!pip install selenium

In [49]:
!pip install bs4


   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ------------- -------------------------- 1/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [bs4]



In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import time
from bs4 import BeautifulSoup
import requests

In [ ]:
base_url = "https://www.myntra.com/"

key_words = ["white shirt","black dress","denim jeans", "summer kurti", "co-ord set", "oversized t-shirt", "sneakers", "blue linen pants", "pink blazer for women","yellow maxi dress"]

path_params = []
for words in key_words:
    path_params.append(words.replace(" ","-"))

In [ ]:
# Setup driver
chrome_driver_path = 'C://Drivers//chromedriver.exe'
chrome_service = Service(chrome_driver_path)
chrome_options = Options()
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Open URL
url = "https://www.myntra.com/men-tshirts"
driver.get(url)
time.sleep(5)

# Loop over pages
pages_cnt=0
while pages_cnt < 3:
    # Scrape current page
    print(f"Scraping page {pages_cnt + 1}...")
    product_elements = driver.find_elements(By.CLASS_NAME, 'product-base')
    for product in product_elements:
        try:
            a_tag = product.find_element(By.TAG_NAME, 'a')
            href = a_tag.get_attribute('href')
            if href:
                print(href)
        except Exception as e:
            print(f"Error: {e}")


    # Try to click the "Next" pagination button using JS
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'pagination-next')
        class_attr = next_button.get_attribute("class")
        
        # Check if 'disabled' or no next button
        if "disabled" in class_attr or not next_button.is_displayed():
            print("No more pages.")
            break

        # Scroll and click using JavaScript
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(4)  # Wait for new page to load
    except Exception as e:
        print(f"Error clicking next: {e}")
        break

    pages_cnt += 1

# Close browser
driver.quit()


Scraping page 1...
https://www.myntra.com/tshirts/nautica/nautica-polo-collar-pure-cotton-t-shirt/31242761/buy
https://www.myntra.com/tshirts/classic+polo/classic-polo-men-striped-polo-collar-pockets-t-shirt/33199683/buy
https://www.myntra.com/tshirts/campus+sutra/campus-sutra-men-self-design-polo-collar-cotton-t-shirt/32193132/buy
https://www.myntra.com/lounge-tshirts/pepe+jeans/pepe-jeans-sleeveless-cotton-lounge-tank-tshirt/28434002/buy
https://www.myntra.com/tshirts/u.s.+polo+assn.+denim+co./us-polo-assn-denim-co-men-typography-printed-round-neck-slim-fit-t-shirt/29908843/buy
https://www.myntra.com/lounge-tshirts/pepe+jeans/pepe-jeans-printed-cotton-basic-vest-8909345003405/34859112/buy
https://www.myntra.com/lounge-tshirts/levis/levis-men-soft-pure-cotton-round-neck-half-sleeve-tshirt/12027436/buy
https://www.myntra.com/tshirts/hektor/hektor-men-self-design-polo-collar-relaxed-fit-t-shirt/34181761/buy
https://www.myntra.com/tshirts/roadster/roadster-men-white--mustard-yellow-colou

Scrapper starts

In [ ]:
# keys:
    # - product_id - ok
    # - keywords_search - ok
    # - Description:
        #   - link - ok
        #   - img_url = []- ok
        #   - originalPrize - ok
        #   - Prize - ok
        #   - Discount - ok
        #   - size -ok
        #   - Rating - ok
        #   - brand - ok
        #   - title - ok

In [18]:
# product-ratingsContainer
# --product-ratingsCount
# product-productMetaInfo
# product-brand
# product-product
# product-price
# product-discountedPrice
# product-discountPercentage

In [54]:
key_words = ["white shirt","black dress","denim jeans", "summer kurti", "co-ord set", "oversized t-shirt", "sneakers", "blue linen pants", "pink blazer for women","yellow maxi dress"]

In [55]:
chrome_driver_path = 'C://Drivers//chromedriver.exe'
chrome_service = Service(chrome_driver_path)
chrome_options = Options()

# Open URL
base_url = "https://www.myntra.com/"

# Loop over pages
pages_cnt=0

In [56]:
def scrapperMyntra(key_word, limit_for_each_keywords,data_myntra_list):
    chrome_service = Service(chrome_driver_path)
    chrome_options = Options()
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    path_param = key_word.replace(" ","-")
    url = base_url + path_param
    # url = "https://www.myntra.com/men-tshirts"
    driver.get(url)
    time.sleep(5)

    product_element = driver.find_element(By.CLASS_NAME, 'results-base')
    product_list = product_element.find_elements(By.CLASS_NAME, 'product-base')
    for scrappeCnt,product in enumerate(product_list): # one chunk of products
        data_myntra = {}
        if scrappeCnt >= limit_for_each_keywords:
            break

        product_id = product.get_attribute('id')

        try:
            rating = product.find_element(By.CLASS_NAME, 'product-ratingsContainer').find_element(By.TAG_NAME,'span').text
            ratingCnt = product.find_element(By.CLASS_NAME, 'product-ratingsContainer').find_elements(By.CLASS_NAME,'product-ratingsCount')[0].text[2:]
        except Exception as e:
            rating = None
            ratingCnt=None
            print(f"Error getting rating: {e}")

        try:
            a_tag = product.find_element(By.TAG_NAME, 'a')
            href = a_tag.get_attribute('href')
            brand = a_tag.find_element(By.CLASS_NAME, 'product-productMetaInfo').find_element(By.CLASS_NAME, 'product-brand').text
            title = a_tag.find_element(By.CLASS_NAME, 'product-productMetaInfo').find_element(By.CLASS_NAME, 'product-product').text
            size = a_tag.find_element(By.CLASS_NAME,'product-sizes').find_element(By.TAG_NAME,'span').get_attribute('innerHTML')
            img_link = a_tag.find_element(By.TAG_NAME,'source').get_attribute('srcset') # as of now only one, it could be list of images link
            try:
                actualPrice = a_tag.find_element(By.CLASS_NAME,'product-strike').text
                discountedPrice = a_tag.find_element(By.CLASS_NAME,'product-discountedPrice').text
                discountPercent = a_tag.find_element(By.CLASS_NAME,'product-discountPercentage').text
            except Exception as e:
                actualPrice = a_tag.find_element(By.CLASS_NAME,'product-price').text
                discountedPrice = actualPrice
                discountPercent = 0
                print(f"No Discounted Prize: {e}")
        except Exception as e:
            print(f"Error: {e}")

        

        data_myntra['product_id'] = product_id
        data_myntra['keywords_search'] = key_word
        data_myntra['description'] = {}
        data_myntra['description']['rating'] = rating
        data_myntra['description']['rating_cnt'] = ratingCnt
        data_myntra['description']['Product_link'] = href
        data_myntra['description']['brand'] = brand
        data_myntra['description']['title'] = title
        data_myntra['description']['img_link'] = img_link
        data_myntra['description']['actualPrice'] = actualPrice
        data_myntra['description']['discountedPrice'] = discountedPrice
        data_myntra['description']['discountPercent'] = discountPercent
        data_myntra['description']['size'] = size

        data_myntra_list.append(data_myntra)
        print(f"scrapped for keyword: {key_word}, scrape count: {scrappeCnt + 1}")
    
    driver.quit()
        

In [57]:
data_myntra = []
limit_for_each_keywords = 10
for key_word in key_words:
    scrapperMyntra(key_word,limit_for_each_keywords,data_myntra)
    time.sleep(5)

Error getting rating: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-ratingsContainer"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff68154e925+77845]
	GetHandleVerifier [0x0x7ff68154e980+77936]
	(No symbol) [0x0x7ff681309cda]
	(No symbol) [0x0x7ff6813606aa]
	(No symbol) [0x0x7ff68136095c]
	(No symbol) [0x0x7ff6813530ac]
	(No symbol) [0x0x7ff68138890f]
	(No symbol) [0x0x7ff681352f76]
	(No symbol) [0x0x7ff681388ae0]
	(No symbol) [0x0x7ff6813b0b07]
	(No symbol) [0x0x7ff6813886a3]
	(No symbol) [0x0x7ff681351791]
	(No symbol) [0x0x7ff681352523]
	GetHandleVerifier [0x0x7ff68182683d+3059501]
	GetHandleVerifier [0x0x7ff681820bfd+3035885]
	GetHandleVerifier [0x0x7ff6818403f0+3164896]
	GetHandleVerifier [0x0x7ff681568c2e+185118]
	GetHandleVerifier [0x0x7ff68157053f+216111

In [58]:
len(data_myntra)

93

In [59]:
import pandas as pd

df_myntra = pd.DataFrame(data_myntra)
# df_myntra.to_csv('myntra_scrapper.csv', index=False)

In [60]:
df_myntra

,product_id,keywords_search,description
0,32393431,white shirt,"{'rating': None, 'rating_cnt': None, 'Product_..."
1,32497662,white shirt,"{'rating': None, 'rating_cnt': None, 'Product_..."
2,20660840,white shirt,"{'rating': '4', 'rating_cnt': '8', 'Product_li..."
3,33526679,white shirt,"{'rating': None, 'rating_cnt': None, 'Product_..."
4,32499277,white shirt,"{'rating': None, 'rating_cnt': None, 'Product_..."
...,...,...,...
88,32507263,yellow maxi dress,"{'rating': None, 'rating_cnt': None, 'Product_..."
89,32670029,yellow maxi dress,"{'rating': '4.3', 'rating_cnt': '50', 'Product..."
90,33317922,yellow maxi dress,"{'rating': '4.3', 'rating_cnt': '20', 'Product..."
91,32727295,yellow maxi dress,"{'rating': None, 'rating_cnt': None, 'Product_..."


In [62]:
df_myntra.groupby('keywords_search').size().reset_index(name='count')

,keywords_search,count
0,black dress,10
1,blue linen pants,10
2,co-ord set,10
3,denim jeans,10
4,oversized t-shirt,10
5,pink blazer for women,10
6,sneakers,10
7,summer kurti,3
8,white shirt,10
9,yellow maxi dress,10


In [63]:
import json
with open('myntra_data.json', 'w') as f:
    json.dump(data_myntra, f, indent=4)

In [5]:
import pandas as pd

In [10]:
df_myntra = pd.read_json('data_myntra.json')

In [11]:
df_myntra['product_id'].groupby(df_myntra['product_id']).size().reset_index(name='count')

,product_id,count
0,375322,1
1,9064593,1
2,12542788,1
3,13999544,1
4,14230962,1
...,...,...
88,35465689,1
89,35585005,1
90,35585009,1
91,35664176,1


Scrapper Ends

In [ ]:
for product in product_list: # one chunk of products
    product_id = product.get_attribute('id')

    try:
        rating = product.find_element(By.CLASS_NAME, 'product-ratingsContainer').find_element(By.TAG_NAME,'span').text
        ratingCnt = product.find_element(By.CLASS_NAME, 'product-ratingsContainer').find_elements(By.CLASS_NAME,'product-ratingsCount')[0].text[2:]
    except Exception as e:
        rating = None
        print(f"Error getting rating: {e}")

    try:
        a_tag = product.find_element(By.TAG_NAME, 'a')
        href = a_tag.get_attribute('href')
        brand = a_tag.find_element(By.CLASS_NAME, 'product-productMetaInfo').find_element(By.CLASS_NAME, 'product-brand').text
        title = a_tag.find_element(By.CLASS_NAME, 'product-productMetaInfo').find_element(By.CLASS_NAME, 'product-product').text
        size = a_tag.find_element(By.CLASS_NAME,'product-sizes').find_element(By.TAG_NAME,'span').get_attribute('innerHTML')
        img_link = a_tag.find_element(By.TAG_NAME,'source').get_attribute('srcset') # as of now only one, it could be list of images link
        try:
            actualPrice = a_tag.find_element(By.CLASS_NAME,'product-strike').text
            discountedPrice = a_tag.find_element(By.CLASS_NAME,'product-discountedPrice').text
            discountPercent = a_tag.find_element(By.CLASS_NAME,'product-discountPercentage').text
        except Exception as e:
            actualPrice = a_tag.find_element(By.CLASS_NAME,'product-price').text
            discountedPrice = actualPrice
            discountPercent = 0
            print(f"No Discounted Prize: {e}")
    except Exception as e:
        print(f"Error: {e}")

    

    data_myntra['product_id'] = product_id
    data_myntra['keywords_search'] = path_param
    data_myntra['description'] = {}
    data_myntra['description']['rating'] = rating
    data_myntra['description']['rating_cnt'] = ratingCnt
    data_myntra['description']['Product_link'] = href
    data_myntra['description']['brand'] = brand
    data_myntra['description']['title'] = title
    data_myntra['description']['img_link'] = img_link
    data_myntra['description']['actualPrice'] = actualPrice
    data_myntra['description']['discountedPrice'] = discountedPrice
    data_myntra['description']['discountPercent'] = discountPercent
    data_myntra['description']['size'] = size

    print(f"scrapped for keyword: {path_param}, scrape count: {scrappeCnt + 1}")

    break

In [145]:
data_myntra

{'product_id': '28542898',
 'keywords_search': 'men-tshirts',
 'description': {'rating': '4.4',
  'rating_cnt': '243',
  'Product_link': 'https://www.myntra.com/tshirts/rare+rabbit/rare-rabbit-men-carmelon-polo-collar-short-sleeves-t-shirt/28542898/buy',
  'brand': 'RARE RABBIT',
  'title': 'Men Carmelon Regular Fit Polo',
  'img_link': 'https://assets.myntassets.com/f_webp,dpr_1.5,q_60,w_210,c_limit,fl_progressive/assets/images/28542898/2024/3/27/42cc45c0-e86c-4b93-89c4-95d944cb8afc1711552634973RARERABBITMenHoodedPocketsT-shirt1.jpg',
  'actualPrice': 'Rs. 3699',
  'discountedPrice': 'Rs. 2034',
  'discountPercent': '(45% OFF)',
  'size': 'XS'}}

In [41]:
len(product_elements)

1

In [44]:
len(product_elements[0].find_elements(By.CLASS_NAME, 'product-base'))

50

In [143]:
product_list[0].find_element(By.TAG_NAME,'a').find_element(By.CLASS_NAME,'product-sizes').find_element(By.TAG_NAME,'span').get_attribute('innerHTML')


'XS'